In [9]:
# Zelle 1: Importiere notwendige Bibliotheken
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches

# Erstelle eine PDF-Datei, in die alle Bilder gespeichert werden sollen
with PdfPages('/Users/nilsgerdes/Projekte/berlin-bike-theft-analysis/output/report.pdf') as pdf:

    # Zelle 2: Lade den Datensatz
    file_path = '/Users/nilsgerdes/Projekte/berlin-bike-theft-analysis/data/Fahrraddiebstahl.csv'  # Ersetze dies mit dem tatsächlichen Pfad
    df = pd.read_csv(file_path, encoding='latin1')

    # Zelle 3: Konvertiere die Datums- und Zeitspalten in datetime-Objekte
    df['TATZEIT_ANFANG_DATUM'] = pd.to_datetime(df['TATZEIT_ANFANG_DATUM'], errors='coerce')  # Fehler werden zu NaT konvertiert
    df['TATZEIT_ENDE_DATUM'] = pd.to_datetime(df['TATZEIT_ENDE_DATUM'], errors='coerce')

    # Zelle 4: Berechne Diebstähle pro Wochentag
    df['weekday'] = df['TATZEIT_ANFANG_DATUM'].dt.weekday
    thefts_per_weekday = df.groupby('weekday').size()

    # Zelle 5: Plot der Diebstähle pro Wochentag
    fig, ax = plt.subplots(figsize=(10, 6))
    thefts_per_weekday.plot(kind='bar', color='lightcoral', ax=ax)
    ax.set_title('Fahrraddiebstähle pro Wochentag')
    ax.set_xlabel('')
    ax.set_ylabel('Anzahl Diebstähle')
    ax.set_xticks(range(7))
    ax.set_xticklabels(['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Samstag', 'Sonntag'], rotation=45)
    ax.set_ylim(2400, 3500)  # Y-Achse spezifizieren
    fig.tight_layout()
    pdf.savefig(fig) 
    plt.close(fig)
    

    # Zelle 6: Berechne Diebstähle pro Monat für die Jahre 2021 und 2022
    df['year'] = df['TATZEIT_ANFANG_DATUM'].dt.year
    df['month'] = df['TATZEIT_ANFANG_DATUM'].dt.month

    # Zelle 7: Filter für 2021 und 2022
    df_2021 = df[df['year'] == 2021]
    df_2022 = df[df['year'] == 2022]

    # Zelle 8: Berechne die Anzahl der Diebstähle pro Monat für 2021 und 2022
    thefts_per_month_2021 = df_2021.groupby('month').size()
    thefts_per_month_2022 = df_2022.groupby('month').size()

    # Zelle 9: Plot der Diebstähle pro Monat für 2021 und 2022
    fig, ax = plt.subplots(figsize=(10, 6))

    # Zelle 10: Plot für 2021
    thefts_per_month_2021.plot(kind='bar', color='lightcoral', width=0.4, position=1, label='2021', ax=ax)

    # Zelle 11: Plot für 2022
    thefts_per_month_2022.plot(kind='bar', color='skyblue', width=0.4, position=0, label='2022', ax=ax)

    # Zelle 12: Plot der Diebstähle pro Monat für 2021 und 2022
    ax.set_title('Fahrraddiebstähle pro Monat für 2021 und 2022')
    ax.set_xlabel('')
    ax.set_ylabel('Anzahl Diebstähle')
    ax.set_xticks(ticks=range(12))
    ax.set_xticklabels(['Januar', 'Februar', 'März', 'April', 'Mai', 'Juni', 'Juli', 'August', 'September', 'Oktober', 'November', 'Dezember'], rotation=45)
    ax.legend()
    ax.set_ylim(0, 3500)
    fig.tight_layout()
    pdf.savefig(fig)
    plt.close(fig)
    

    # Zelle 13: Liste der Jahre im Datensatz
    unique_years = df['year'].unique()
    # Gruppiere nach Fahrradtyp und zähle die Anzahl
    bike_counts = df['ART_DES_FAHRRADS'].value_counts()

    # Zelle 14: Plot der Diebstähle pro Fahrradtyp
    fig, ax = plt.subplots(figsize=(10, 6))
    bike_counts.plot(kind='bar', color='skyblue', edgecolor='black')
    ax.set_title('Anzahl der Diebstähle pro Fahrradtyp')
    ax.set_xlabel('')
    ax.set_ylabel('Anzahl der Diebstähle')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    fig.tight_layout()
    pdf.savefig(fig) 
    plt.close(fig)

    # Zelle 15: Gruppieren: Anzahl und Durchschnittsschaden pro LOR
    lor_stats = df.groupby('LOR').agg(
    anzahl=('SCHADENSHOEHE', 'count'),
    durchschnitt=('SCHADENSHOEHE', 'mean')
    )

    # Zelle 16: Filtern: Nur Orte mit mehr als 100 Diebstählen
    lor_over_100 = lor_stats[lor_stats['anzahl'] > 125].sort_values('anzahl', ascending=False)

    # Zelle 17: Plot der LORs mit mehr als 100 Diebstählen
    fig, ax = plt.subplots(figsize=(10, 6))
    lor_over_100['anzahl'].plot(kind='bar', figsize=(10,6), color='steelblue')
    ax.set_title('LORs mit mehr als 100 Diebstählen')
    ax.set_xlabel('')
    ax.set_ylabel('Anzahl der Diebstähle')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    fig.tight_layout()
    pdf.savefig(fig) 
    plt.close(fig)


    # Zelle 18: Filtern: Schaden zwischen 1000 und 1200 €, mind. 100 Diebstähle
    filtered_lor = lor_stats[
    (lor_stats['anzahl'] > 100) &
    (lor_stats['durchschnitt'] >= 1000) &
    (lor_stats['durchschnitt'] <= 1200)
    ].sort_values('durchschnitt', ascending=False)

    # Zelle 19: Plot der LORs mit durchschnittlichem Schaden zwischen 1000 € und 1150 € und nur >100 Diebstähle
    fig, ax = plt.subplots(figsize=(10, 6))
    filtered_lor['durchschnitt'].plot(kind='bar', figsize=(10,6), color='orange')
    ax.set_title('LORs mit durchschnittlichem Schaden zwischen 1000 € und 1150 € (nur >100 Diebstähle)')
    ax.set_xlabel('')
    ax.set_ylabel('Durchschnittlicher Schaden (€)')
    ax.set_ylim(1000, 1150)  # Eingrenzung der Y-Achse
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    fig.tight_layout()
    pdf.savefig(fig) 
    plt.close(fig)

    # Zelle 20:  Lade zweiten Datensatz
    shp_path = "/Users/nilsgerdes/Projekte/berlin-bike-theft-analysis/data/shapefiles/LOR_2023-01-01_PLR_EPSG_25833_nur_ID.shp"
    gdf_lor = gpd.read_file(shp_path)


    # Zelle 21: Kürze die PLR_IDs auf 7-stellig, passend zu den LORs aus der CSV
    gdf_lor['PLR_ID_clean'] = gdf_lor['PLR_ID'].astype(str).str[-7:]
    df['LOR_clean'] = df['LOR'].astype(str).str.zfill(7)  # stellt sicher, dass LOR auch 7-stellig ist

    # Zelle 22: Merge der Daten nach den bereinigten IDs
    merged = gdf_lor.merge(df, left_on='PLR_ID_clean', right_on='LOR_clean', how='left')

    # Zelle 23: Gruppieren der Diebstähle nach LOR
    lor_counts = df.groupby('LOR_clean').size().reset_index(name='anzahl_diebstaehle')

    # Zelle 24: Join mit GeoDataFrame für Anzahl der Diebstähle
    gdf_counts_joined = gdf_lor.merge(lor_counts, left_on='PLR_ID_clean', right_on='LOR_clean', how='left')

    # Zelle 25: NaN durch 0 ersetzen (wenn in einem LOR kein Diebstahl vorkam)
    gdf_counts_joined['anzahl_diebstaehle'] = gdf_counts_joined['anzahl_diebstaehle'].fillna(0)

    # Zelle 26: Entfernen von 0er Schadenshöhen
    df_clean_damage = df[df['SCHADENSHOEHE'] != 0]  
    lor_damage = df_clean_damage.groupby('LOR_clean')['SCHADENSHOEHE'].sum().reset_index(name='schadenhoehe')

    # Zelle 27: Join mit GeoDataFrame für Schadenshöhe
    gdf_damage_joined = gdf_lor.merge(lor_damage, left_on='PLR_ID_clean', right_on='LOR_clean', how='left')

    # Zelle 28: NaN durch 0 ersetzen (wenn kein Schaden in einem LOR vorkam)
    gdf_damage_joined['schadenhoehe'] = gdf_damage_joined['schadenhoehe'].fillna(0)
    
    # Zelle 29: Merge der Diebstahl- und Schadensdaten
    gdf_kategorien = gdf_counts_joined.merge(gdf_damage_joined[['PLR_ID_clean', 'schadenhoehe']], 
                                          on='PLR_ID_clean', 
                                          how='left')
     # Zelle 30: Berechne den Schaden pro Diebstahl
    gdf_kategorien['schaden_pro_diebstahl'] = gdf_kategorien['schadenhoehe'] / gdf_kategorien['anzahl_diebstaehle']
  

    # Zelle 29: Erstellen der Karten für Anzahl der Diebstähle pro LOR

    fig, ax = plt.subplots(1, 1, figsize=(12, 12))
    gdf_counts_joined.plot(column='anzahl_diebstaehle',
                       cmap='OrRd',
                       linewidth=0.8,
                       edgecolor='0.8',
                       legend=True,
                       ax=ax)
    ax.set_title('Fahrraddiebstähle pro LOR', fontsize=16)
    ax.axis('off')
    plt.tight_layout()
    pdf.savefig(fig) 
    plt.close(fig)

    # Zelle 30: Erstellen der Karte für Schadenshöhe pro LOR
    fig, ax = plt.subplots(1, 1, figsize=(12, 12))
    gdf_damage_joined.plot(column='schadenhoehe',
                       cmap='YlGnBu',
                       linewidth=0.8,
                       edgecolor='0.8',
                       legend=True,
                       ax=ax)
    ax.set_title('Schadenshöhe pro LOR', fontsize=16)
    ax.axis('off')
    plt.tight_layout()
    pdf.savefig(fig) 
    plt.close(fig)

    # Zelle 31: Berechne den Durchschnitt der Schadenshöhe pro LOR
    lor_damage_avg = df_clean_damage.groupby('LOR_clean')['SCHADENSHOEHE'].mean().reset_index(name='durchschnittliche_schadenhoehe')

    # Zelle 32: Join mit GeoDataFrame für die durchschnittliche Schadenshöhe
    gdf_avg_damage_joined = gdf_lor.merge(lor_damage_avg, left_on='PLR_ID_clean', right_on='LOR_clean', how='left')

    # Zelle 33: NaN durch 0 ersetzen, falls es für bestimmte LORs keine Schadenshöhe gibt
    gdf_avg_damage_joined['durchschnittliche_schadenhoehe'] = gdf_avg_damage_joined['durchschnittliche_schadenhoehe'].fillna(0)

    # Zelle 34: Erstellen der Karte für durchschnittliche Schadenshöhe
    fig, ax = plt.subplots(1, 1, figsize=(12, 12))
    gdf_avg_damage_joined.plot(column='durchschnittliche_schadenhoehe',
                           cmap='YlGnBu',  # Farbskala für Schadenshöhe
                           linewidth=0.8,
                           edgecolor='0.8',
                           legend=True,
                           ax=ax)
    ax.set_title('Durchschnittliche Schadenshöhe pro LOR', fontsize=16)
    ax.axis('off')
    plt.tight_layout()
    pdf.savefig(fig) 
    plt.close(fig)

    # Zelle 35: Gruppieren nach LOR und aggregieren
    lor_counts = df.groupby('LOR_clean').size().reset_index(name='anzahl_diebstaehle')
    lor_damage = df.groupby('LOR_clean')['SCHADENSHOEHE'].sum().reset_index(name='schadenhoehe')

    # Zelle 36: Mergen der beiden Aggregationen
    lor_summary = lor_counts.merge(lor_damage, on='LOR_clean', how='outer')
    lor_summary['schadenhoehe'] = lor_summary['schadenhoehe'].fillna(0)
    lor_summary['anzahl_diebstaehle'] = lor_summary['anzahl_diebstaehle'].fillna(0)

    # Zelle 37: Durchschnittlicher Schaden pro Diebstahl
    lor_summary['schaden_pro_diebstahl'] = lor_summary.apply(
        lambda row: row['schadenhoehe'] / row['anzahl_diebstaehle'] if row['anzahl_diebstaehle'] > 0 else 0,
        axis=1
    )
    
    # Zellen 38: Dynamische Schwellen berechnen
    anzahl_schwelle = gdf_kategorien['anzahl_diebstaehle'].quantile(0.66)
    schaden_schwelle = gdf_kategorien['schaden_pro_diebstahl'].quantile(0.66)
    # Zellen 39 Lege einzele Kategorien dynamisch fest
    def kategorie_dynamisch(row):
        viele = row['anzahl_diebstaehle'] > anzahl_schwelle
        teuer = row['schaden_pro_diebstahl'] > schaden_schwelle

        if viele and teuer:
            return "Viele & teure Diebstähle"
        elif viele and not teuer:
            return "Viele & günstige Diebstähle"
        elif not viele and teuer:
            return "Wenige & teure Diebstähle"
        elif not viele and not teuer:
            return "Wenige & günstige Diebstähle"
        else:
            return "Unklar"  # sollte nicht auftreten
    
    # Zelle 40: Kategorien zuweisen
    gdf_kategorien['Kategorie'] = gdf_kategorien.apply(kategorie_dynamisch, axis=1)

    farben = {
        "Viele & teure Diebstähle": "#d7191c",      # Rot
        "Viele & günstige Diebstähle": "#fdae61",   # Orange
        "Wenige & teure Diebstähle": "#2b83ba",     # Blau
        "Wenige & günstige Diebstähle": "#abdda4",  # Grün
    }

    # Zelle 41: Plot erstellen für dynamische Zuweisung nach Kategorien Schadenshöhe und Anzahl Diebstähle
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    gdf_kategorien.plot(
        column='Kategorie',
        ax=ax,
        color=gdf_kategorien['Kategorie'].map(farben),
        edgecolor='black'
    )

    # Zelle 42: Manuelle Legende erstellen
    legende_patches = [
        mpatches.Patch(color=color, label=label)
        for label, color in farben.items()
    ]
    ax.legend(handles=legende_patches, loc='lower left', title='Kategorie', fontsize=10)
    ax.set_title("Fahrraddiebstahl-Kategorien pro LOR (dynamisch)", fontsize=14)
    ax.axis('off')
    fig.tight_layout()
    pdf.savefig(fig) 
    plt.close(fig)



/Users/nilsgerdes/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '21.02.2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/Users/nilsgerdes/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '22.02.2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/Users/nilsgerdes/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '19.02.2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/Users/nilsgerdes/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning:

/var/folders/r9/kkc694tn4b303shyztlq8fj00000gn/T/ipykernel_54680/2092441810.py:264: UserWarning: Only specify one of 'column' or 'color'. Using 'color'.
  gdf_kategorien.plot(
